In [1]:
import pandas as pd
import numpy as np
from rsome import ro
from rsome import grb_solver as grb

In [2]:
data1 = pd.read_csv('menu.csv')

data1['Total'] = 100

print( data1.head() )

            Ingredient Ingredient_type  Price  Serving_size  COGS  \
0  Red & White Cabbage   Standard Base    NaN            90  6.45   
1              Romaine   Standard Base    NaN           120  3.10   
2                 Kale    Premium Base    NaN            90  5.00   
3         Baby Spinach    Premium Base    NaN            90  1.25   
4       Wholemeal Wrap            Wrap    NaN           100  3.55   

   per (g, pcs, unit)  COGS_per_serving  \
0              1100.0              0.53   
1               200.0              1.86   
2               120.0              3.75   
3               200.0              0.56   
4               360.0              0.99   

   Adjusted_COGS_per_serving (if needed)  Calories  Carbohydrates  ...  \
0                                    NaN        26            3.4  ...   
1                                   0.86        20            4.0  ...   
2                                   1.75        32            4.0  ...   
3                             

In [3]:
# opening_hours_input = 10

# num_of_salads = opening_hours_input*60

# estimated_total_servings = num_of_salads * (2 + 7 + 2 + 2)

# servings_per_ingredient_per_day = round(estimated_total_servings/len(data1))

In [4]:
header1 = data1.columns

In [14]:
def get_LOP_results(user_input):
    print(user_input)
    print(total)
    # Preparing an optimization model
    model = ro.Model('Salad selector model')

    n = len(ingredient)

    # servings of ingredient in salad, binary variable
    x = model.dvar(n, vtype='B')

    # s for standard base
    s = model.dvar()

    # t for premium base
    t = model.dvar()

    # Setting objective
    model.max( (9.9*s + 11.9*t + sum(x[i]*price[i] for i in range(n) if ingredient_type[i] in ['Premium Topping']) ) - (sum(x[i]*cost[i] for i in range(n))) )

    # Setting constraints
    
    # ingredients resource constraints
    model.st( sum(x[i] for i in range(n) if total[i] == 0) == 0 )

    # Selecting standard/premium base
    model.st( sum( x[i] for i in range(n) if ingredient_type[i] in ['Standard Base', 'Wrap', 'Grain Bowl'] ) == s )
    model.st( 0 <= s <= 1 )
    model.st( sum( x[i] for i in range(n) if ingredient_type[i] in ['Premium Base'] ) == t )
    model.st( 0 <= t <= 1 )
    model.st( s + t == 1 )

    # selecting 7 standard toppings
    model.st(sum( x[i] for i in range(n) if ingredient_type[i] in ['Standard Topping'] ) == 7 )

    # selecting 2 dressings
    model.st(sum( x[i] for i in range(n) if ingredient_type[i] in ['Dressing (Asian)', 'Dressing (Western)'] ) == 2 )

    # nutritional constraints based on user input
    nutrition_list = [calories, carbs, protein, fat, sugar]
    for j in range(len(nutrition_list)):
        nutri =  nutrition_list[j]
        model.st( user_input["min_nutrition"][j] <= sum(x[i]*nutri[i] for i in range(n)) )
        model.st( sum(x[i]*nutri[i] for i in range(n) ) <= user_input["max_nutrition"][j] )
        
    # budget constraint based on user input
    model.st( (9.9*s + 11.9*t + sum(x[i]*price[i] for i in range(n) if ingredient_type[i] in ['Premium Topping'])) <= user_input["budget"])

    # select num of premium
    model.st(sum( x[i] for i in range(n) if ingredient_type[i] in ['Premium Topping'] ) <= user_input["max_num_of_premium_toppings"] )

    #dietary requirements constraints
    reqs = [vegan, vegetarian, gluten, dairy, nuts, spicy]
    for k in range(len(user_input["dietary_req"])):
        req_type = reqs[k]
        if user_input["dietary_req"][k] == 0:
            model.st( sum(x[i] for i in range(n) if req_type[i] == 1) == 0 )

    # carbon emissions constraints


    # non negativity
    model.st( x >= 0 )

    # Solve model
    model.solve(grb)
    res = x.get()
#     print(x.get())
#     print(s.get())
#     print(t.get())
#     print(model.get())

    print(sum(res[i] for i in range(n) if total[i] == 0))

    for i in range(len(res)):
        if res[i] > 0:
            print(ingredient[i])
            if total[i] > 0:
                total[i] -= 1
                
    for j in range(len(nutrition_list)):
        nutri =  nutrition_list[j]        
        print(sum(res[i]*nutri[i] for i in range(n) ))       

    
    return 

In [16]:
num_of_salads = 0
user_input = {
    "min_nutrition": np.array([450, 30, 20, 0, 0]), # calories, carbs, protein, fat, sugar
    "max_nutrition": np.array([500, 500, 30, 10, 10]),
    "budget" : 17,
    "max_num_of_premium_toppings": 3,
    "dietary_req": [1,1,1,1,1,1] # vegan, vegetarian, gluten, dairy, nuts, spicy
}

data = data1.values

ingredient, ingredient_type, price, cost, calories, carbs, protein, fat, sugar =\
data[:,0], data[:,1], data[:,2], data[:,6], data[:,8], data[:,9], data[:,10], data[:,11], data[:,12]

vegan, vegetarian, gluten, dairy, nuts, spicy =\
data[:,-9], data[:,-8], data[:,-7], data[:,-6], data[:,-5], data[:,-4]

total = data[:,-1]
total[-1] = 0


for i in range(200):
    get_LOP_results(user_input)

{'min_nutrition': array([450,  30,  20,   0,   0]), 'max_nutrition': array([500, 500,  30,  10,  10]), 'budget': 17, 'max_num_of_premium_toppings': 3, 'dietary_req': [1, 1, 1, 1, 1, 1]}
[100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 0]
Being solved by Gurobi...
Solution status: 2
Running time: 0.0259s
0.0
Quinoa
Green Apple
Soba Noodles
Potato
Black Beans
French Beans
Red Onions
Carrot
Thai Asparagus
Smoked Salmon
Whole Eggs
Salt & Pepper
Tabbasco Sauce
465.0
67.5
28.599999999999994
10.0
6.999999999999998
{'min_nutrition': array([450,  30,  20,   0,   0]), 'max_nutrition': array([500, 500,  30,  10,  10]), 'budget': 17, 'max_num_of_premium_toppings': 3, 'dietary_req': [1, 1, 1, 1, 1, 1

Being solved by Gurobi...
Solution status: 2
Running time: 0.0369s
0.0
Quinoa
Green Apple
Soba Noodles
Potato
Black Beans
French Beans
Red Onions
Carrot
Thai Asparagus
Smoked Salmon
Whole Eggs
Salt & Pepper
Tabbasco Sauce
465.0
67.5
28.599999999999994
10.0
6.999999999999998
{'min_nutrition': array([450,  30,  20,   0,   0]), 'max_nutrition': array([500, 500,  30,  10,  10]), 'budget': 17, 'max_num_of_premium_toppings': 3, 'dietary_req': [1, 1, 1, 1, 1, 1]}
[100 100 100 100 100 100 100 100 100 89 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 89 100 100 89 100 89 89 100 100 100 100 89
 100 89 100 100 100 89 100 100 100 100 89 100 100 100 100 100 100 100 89
 100 100 100 100 100 89 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 89 100 100 89 0]
Being solved by Gurobi...
Solution status: 2
Running time: 0.0249s
0.0
Quinoa
Green Apple
Soba Noodles
Potato
Black Beans
French Beans
Red Onions
Carrot
Thai Asparagus
Smoked Salmon
Whole Eggs
Sal

Being solved by Gurobi...
Solution status: 2
Running time: 0.0288s
0.0
Quinoa
Green Apple
Soba Noodles
Potato
Black Beans
French Beans
Red Onions
Carrot
Thai Asparagus
Smoked Salmon
Whole Eggs
Salt & Pepper
Tabbasco Sauce
465.0
67.5
28.599999999999994
10.0
6.999999999999998
{'min_nutrition': array([450,  30,  20,   0,   0]), 'max_nutrition': array([500, 500,  30,  10,  10]), 'budget': 17, 'max_num_of_premium_toppings': 3, 'dietary_req': [1, 1, 1, 1, 1, 1]}
[100 100 100 100 100 100 100 100 100 78 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 78 100 100 78 100 78 78 100 100 100 100 78
 100 78 100 100 100 78 100 100 100 100 78 100 100 100 100 100 100 100 78
 100 100 100 100 100 78 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 78 100 100 78 0]
Being solved by Gurobi...
Solution status: 2
Running time: 0.0289s
0.0
Quinoa
Green Apple
Soba Noodles
Potato
Black Beans
French Beans
Red Onions
Carrot
Thai Asparagus
Smoked Salmon
Whole Eggs
Sal

Being solved by Gurobi...
Solution status: 2
Running time: 0.0389s
0.0
Quinoa
Green Apple
Soba Noodles
Potato
Black Beans
French Beans
Red Onions
Carrot
Thai Asparagus
Smoked Salmon
Whole Eggs
Salt & Pepper
Tabbasco Sauce
465.0
67.5
28.599999999999994
10.0
6.999999999999998
{'min_nutrition': array([450,  30,  20,   0,   0]), 'max_nutrition': array([500, 500,  30,  10,  10]), 'budget': 17, 'max_num_of_premium_toppings': 3, 'dietary_req': [1, 1, 1, 1, 1, 1]}
[100 100 100 100 100 100 100 100 100 67 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 67 100 100 67 100 67 67 100 100 100 100 67
 100 67 100 100 100 67 100 100 100 100 67 100 100 100 100 100 100 100 67
 100 100 100 100 100 67 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 67 100 100 67 0]
Being solved by Gurobi...
Solution status: 2
Running time: 0.0328s
0.0
Quinoa
Green Apple
Soba Noodles
Potato
Black Beans
French Beans
Red Onions
Carrot
Thai Asparagus
Smoked Salmon
Whole Eggs
Sal

Being solved by Gurobi...
Solution status: 2
Running time: 0.0329s
0.0
Quinoa
Green Apple
Soba Noodles
Potato
Black Beans
French Beans
Red Onions
Carrot
Thai Asparagus
Smoked Salmon
Whole Eggs
Salt & Pepper
Tabbasco Sauce
465.0
67.5
28.599999999999994
10.0
6.999999999999998
{'min_nutrition': array([450,  30,  20,   0,   0]), 'max_nutrition': array([500, 500,  30,  10,  10]), 'budget': 17, 'max_num_of_premium_toppings': 3, 'dietary_req': [1, 1, 1, 1, 1, 1]}
[100 100 100 100 100 100 100 100 100 56 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 56 100 100 56 100 56 56 100 100 100 100 56
 100 56 100 100 100 56 100 100 100 100 56 100 100 100 100 100 100 100 56
 100 100 100 100 100 56 100 100 100 100 100 100 100 100 100 100 100 100
 100 100 100 100 100 100 100 100 100 56 100 100 56 0]
Being solved by Gurobi...
Solution status: 2
Running time: 0.0328s
0.0
Quinoa
Green Apple
Soba Noodles
Potato
Black Beans
French Beans
Red Onions
Carrot
Thai Asparagus
Smoked Salmon
Whole Eggs
Sal

...

Being solved by Gurobi...
Solution status: 2
Running time: 0.0289s
0.0
Tomato Wrap
Pea Sprouts
Sesame Seeds
Furikake
Lime Wedge
Fresh Herbs
Roasted Pumpkins
Cucumbers
Seared Tuna
Thai Turmeric
Smoked Pimento
476.0
73.30000000000001
27.2
6.599999999999998
9.899999999999999
{'min_nutrition': array([450,  30,  20,   0,   0]), 'max_nutrition': array([500, 500,  30,  10,  10]), 'budget': 17, 'max_num_of_premium_toppings': 3, 'dietary_req': [1, 1, 1, 1, 1, 1]}
[100 100 100 100 100 100 100 46 100 0 46 100 100 100 46 46 100 100 46 100
 46 100 100 100 100 0 100 100 0 46 0 0 100 100 100 100 0 100 0 100 100 46
 0 100 100 100 100 0 100 100 100 100 100 100 46 0 100 100 100 100 100 0
 100 100 100 100 100 100 100 100 100 100 100 46 100 100 100 100 100 100
 100 100 100 0 46 100 0 0]
Being solved by Gurobi...
Solution status: 2
Running time: 0.0328s
0.0
Tomato Wrap
Pea Sprouts
Sesame Seeds
Furikake
Lime Wedge
Fresh Herbs
Roasted Pumpkins
Cucumbers
Seared Tuna
Thai Turmeric
Smoked Pimento
476.0
73.30000

Being solved by Gurobi...
Solution status: 2
Running time: 0.0287s
0.0
Tomato Wrap
Pea Sprouts
Sesame Seeds
Furikake
Lime Wedge
Fresh Herbs
Roasted Pumpkins
Cucumbers
Seared Tuna
Thai Turmeric
Smoked Pimento
476.0
73.30000000000001
27.2
6.599999999999998
9.899999999999999
{'min_nutrition': array([450,  30,  20,   0,   0]), 'max_nutrition': array([500, 500,  30,  10,  10]), 'budget': 17, 'max_num_of_premium_toppings': 3, 'dietary_req': [1, 1, 1, 1, 1, 1]}
[100 100 100 100 100 100 100 35 100 0 35 100 100 100 35 35 100 100 35 100
 35 100 100 100 100 0 100 100 0 35 0 0 100 100 100 100 0 100 0 100 100 35
 0 100 100 100 100 0 100 100 100 100 100 100 35 0 100 100 100 100 100 0
 100 100 100 100 100 100 100 100 100 100 100 35 100 100 100 100 100 100
 100 100 100 0 35 100 0 0]
Being solved by Gurobi...
Solution status: 2
Running time: 0.0309s
0.0
Tomato Wrap
Pea Sprouts
Sesame Seeds
Furikake
Lime Wedge
Fresh Herbs
Roasted Pumpkins
Cucumbers
Seared Tuna
Thai Turmeric
Smoked Pimento
476.0
73.30000

Being solved by Gurobi...
Solution status: 2
Running time: 0.0419s
0.0
Tomato Wrap
Pea Sprouts
Sesame Seeds
Furikake
Lime Wedge
Fresh Herbs
Roasted Pumpkins
Cucumbers
Seared Tuna
Thai Turmeric
Smoked Pimento
476.0
73.30000000000001
27.2
6.599999999999998
9.899999999999999
{'min_nutrition': array([450,  30,  20,   0,   0]), 'max_nutrition': array([500, 500,  30,  10,  10]), 'budget': 17, 'max_num_of_premium_toppings': 3, 'dietary_req': [1, 1, 1, 1, 1, 1]}
[100 100 100 100 100 100 100 24 100 0 24 100 100 100 24 24 100 100 24 100
 24 100 100 100 100 0 100 100 0 24 0 0 100 100 100 100 0 100 0 100 100 24
 0 100 100 100 100 0 100 100 100 100 100 100 24 0 100 100 100 100 100 0
 100 100 100 100 100 100 100 100 100 100 100 24 100 100 100 100 100 100
 100 100 100 0 24 100 0 0]
Being solved by Gurobi...
Solution status: 2
Running time: 0.0278s
0.0
Tomato Wrap
Pea Sprouts
Sesame Seeds
Furikake
Lime Wedge
Fresh Herbs
Roasted Pumpkins
Cucumbers
Seared Tuna
Thai Turmeric
Smoked Pimento
476.0
73.30000

Being solved by Gurobi...
Solution status: 2
Running time: 0.0487s
0.0
Tomato Wrap
Pea Sprouts
Sesame Seeds
Furikake
Lime Wedge
Fresh Herbs
Roasted Pumpkins
Cucumbers
Seared Tuna
Thai Turmeric
Smoked Pimento
476.0
73.30000000000001
27.2
6.599999999999998
9.899999999999999
{'min_nutrition': array([450,  30,  20,   0,   0]), 'max_nutrition': array([500, 500,  30,  10,  10]), 'budget': 17, 'max_num_of_premium_toppings': 3, 'dietary_req': [1, 1, 1, 1, 1, 1]}
[100 100 100 100 100 100 100 13 100 0 13 100 100 100 13 13 100 100 13 100
 13 100 100 100 100 0 100 100 0 13 0 0 100 100 100 100 0 100 0 100 100 13
 0 100 100 100 100 0 100 100 100 100 100 100 13 0 100 100 100 100 100 0
 100 100 100 100 100 100 100 100 100 100 100 13 100 100 100 100 100 100
 100 100 100 0 13 100 0 0]
Being solved by Gurobi...
Solution status: 2
Running time: 0.0429s
0.0
Tomato Wrap
Pea Sprouts
Sesame Seeds
Furikake
Lime Wedge
Fresh Herbs
Roasted Pumpkins
Cucumbers
Seared Tuna
Thai Turmeric
Smoked Pimento
476.0
73.30000

Being solved by Gurobi...
Solution status: 2
Running time: 0.0297s
0.0
Tomato Wrap
Pea Sprouts
Sesame Seeds
Furikake
Lime Wedge
Fresh Herbs
Roasted Pumpkins
Cucumbers
Seared Tuna
Thai Turmeric
Smoked Pimento
476.0
73.30000000000001
27.2
6.599999999999998
9.899999999999999
{'min_nutrition': array([450,  30,  20,   0,   0]), 'max_nutrition': array([500, 500,  30,  10,  10]), 'budget': 17, 'max_num_of_premium_toppings': 3, 'dietary_req': [1, 1, 1, 1, 1, 1]}
[100 100 100 100 100 100 100 2 100 0 2 100 100 100 2 2 100 100 2 100 2 100
 100 100 100 0 100 100 0 2 0 0 100 100 100 100 0 100 0 100 100 2 0 100 100
 100 100 0 100 100 100 100 100 100 2 0 100 100 100 100 100 0 100 100 100
 100 100 100 100 100 100 100 100 2 100 100 100 100 100 100 100 100 100 0 2
 100 0 0]
Being solved by Gurobi...
Solution status: 2
Running time: 0.0289s
0.0
Tomato Wrap
Pea Sprouts
Sesame Seeds
Furikake
Lime Wedge
Fresh Herbs
Roasted Pumpkins
Cucumbers
Seared Tuna
Thai Turmeric
Smoked Pimento
476.0
73.30000000000001
2